## Basic examples of tensorSB


In [1]:
%load_ext autoreload
%autoreload 2


### Import tensorSB

There are three types of backends: Torch, NumPy, and CuPy.

Once a backend is selected, all tensors will be created using that backend.
In most cases, you do not need to explicitly import ```Torch/NumPy/CuPy``` modules, since tensor creation and manipulation functions are abstracted and exposed through tensorSB.

The backend type is controlled by the ```BACKEND_TYPE``` environment variable.

In [9]:
import os
import importlib
os.environ["BACKEND_TYPE"]="torch"
# os.environ["BACKEND_TYPE"]="numpy"
# os.environ["BACKEND_TYPE"]="cupy"
try:
    tn.reset_backend()
except:
    pass
import tensorSB as tn
importlib.reload(tn)

<module 'tensorSB' from '/home/subini0213/cuQuantum/tensorSB/src/tensorSB/__init__.py'>

### Backend functions

In [10]:
t1 = tn.backend.get_rand([1,2,3])
t2 = tn.backend.get_rand([1,2,5])
t3 = tn.backend.get_rand([2,3,5,7],"uniform")
t12 = tn.backend.cat((t1,t2),2)
t1p = tn.backend.permute(t1,[1,0,2])
t3r = tn.backend.reshape(t3,[6,35])
eye7 = tn.backend.eye(7)
t3rd = tn.tensor.Hconj(t3r)
I=tn.tensor.get_identity(t1,1,t2,2,[2,0,1])
I2 = tn.tensor.get_identity(t1,1)


# print
print(f"t1.shape : {t1.shape}")
print(f"t2.shape : {t2.shape}")
print(f"t3.shape : {t3.shape}")
print(f"t12.shape : {t12.shape}")
print(f"t1p.shape : {t1p.shape}")
print(f"t3r.shape : {t3r.shape}")
print(f"eye7.shape : {eye7.shape}")
print(f"t3rd.shape : {t3rd.shape}")
print(f"I.shape : {I.shape}")
print(f"I2.shape : {I2.shape}")

t1.shape : torch.Size([1, 2, 3])
t2.shape : torch.Size([1, 2, 5])
t3.shape : torch.Size([2, 3, 5, 7])
t12.shape : torch.Size([1, 2, 8])
t1p.shape : torch.Size([2, 1, 3])
t3r.shape : torch.Size([6, 35])
eye7.shape : torch.Size([7, 7])
t3rd.shape : torch.Size([35, 6])
I.shape : torch.Size([10, 2, 5])
I2.shape : torch.Size([2, 2])


### Pauli Operators

```S,I = tn.tensor.get_local_space('Spin', s)```

```S``` is rank-3 tensor operator, which is spin operator of spin-s boson/fermion, and ```I``` is rank-2 identity tensor with same Hilbert space.

 ```S[:,:,0], S[:,:,0], S[:,:,0]``` correspond to  $\frac{1}{\sqrt{2}}S_{+}, S_z, \frac{1}{\sqrt{2}}S_{-}$

In [11]:
S,I = tn.tensor.get_local_space('Spin', 1/2)
print(S[:,:,0])

tensor([[0.0000, 0.7071],
        [0.0000, 0.0000]])


### Contraction

- **`library="einsum"`**  
  Calls `einsum` from the selected backend (`torch.einsum`, `numpy.einsum`, or `cupy.einsum`).  
  This option is generally the most efficient for contractions involving **two tensors**.

- **`library="cuquantum"`**  
  Uses `cuquantum.tensornet.contract`, which automatically optimizes the contraction path.  
  Recommended when contracting **three or more tensors**, and the optimal path is not determined.   
  This option is not available without a cuda environment.

In [17]:

d = 20
t1 = tn.backend.get_rand([10,12,d]) # abi
t2 = tn.backend.get_rand([11,13,d,d]) # cjid
t3 = tn.backend.get_rand([12,15,13]) #bej

# Default: einsum
A = tn.tensor.contract('abi,cjid->abcjd', t1, t2)
A = tn.tensor.contract('abcjd,bej->acde',A,t3)

# CuQuantum
# A = tn.tensor.contract('abi,cjid,bej->acde', t1, t2, t3, library='cuquantum')

print(A.shape)

torch.Size([10, 11, 20, 15])


### Decompositions

Singular Value Decomposition (SVD) and QR decomposition are available.  
To run without a CUDA environment, specify `use_cuda=False`.


In [19]:
t = tn.backend.get_rand([1000, 1000, 6])

u, s, v = tn.tensor.svd('ijx->ikx,kj',t,use_cuda=False)
# q ,r = tn.tensor.qr('ijx->ikx,kj',t)

print(u.shape)
print(tn.backend.diag(s).shape)
print(v.shape)

torch.Size([1000, 1000, 6])
torch.Size([1000, 1000])
torch.Size([1000, 1000])
